# Multi-voxel pattern analysis

This week we will use a popular python library to do simple classification of a fake data set, and of some real fMRI data. My original intent was to implement a linear discriminant from scratch, but... that's a bit more math than we have time for in this session. So instead, we will explore [`sklearn`](http://scikit-learn.org/stable/) a bit, and show you how to make use of python libraries to solve your problems.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel
import h5py
import os

# For to fancy
import fakedata as fd
import cortex as cx

# For to machine learn
from sklearn import svm
from sklearn import discriminant_analysis as da

%matplotlib inline

# Simple classifier with fake data
First, to perform classification at all, we need some data. We need an `X` variable, consisting of the features to use for classification, and a `Y` variable, consisting of class labels. This should be pretty easy to generate, given what we have learned so far in class. Your first task is to write code to generate the plot below!

<img src="TwoClasses.png">

You should write your code in such a way that it is easy to make the distributions of points closer to each other / slightly overlapping. Also, you should divide your data into a training / testing set!

In [ ]:
# Answer



In [ ]:
# Define 2D values for class 1
n = 500 # number of exemplars per class
#r = 0.6
mu_c1 = np.array([-0.5, 0.3])
mu_c2 = np.array([0.5, -0.3])
Xt, Xv, Yt, Yv = generate_data(n, mu_c1, mu_c2) #, cov_c1=r, cov_c2=r)

plot_classes(Xt[Yt==1], Xt[Yt==2])

# sklearn!

Now we will use the sklearn (scikit-learn) machine learning library to classify the points into each class! 

In [ ]:
# Get SVM classifier base object
svmclf = svm.LinearSVC()
# Fit SVM classifier to data
_ = svmclf.fit(Xt, Yt)
# Predict new values
Ypred_svm = svmclf.predict(Xv)

In [ ]:
ldaclf = da.LinearDiscriminantAnalysis()
# Fit LDA classifier to data
_ = ldaclf.fit(Xt, Yt)
# Predict new responses w/ LDA classifier
Ypred_lda = ldaclf.predict(Xv)

> How should we test accuracy?

In [ ]:
# Answer


# Show decision boundary 
### But first, a little basic algebra review...

Perhaps the most familiar form of a line is:

$y = mx + b$

...but a more general form for a line is the following: (this can take x, y, z,... arbitrarily many variables, each with a constant). 

$0 = Ax + By + C$

This form of a line is what is returned by the classification algorithms. To put this into the more familiar form (for a 2D line only), we can do the following:

$m = -A/B$ 

and

$b = -C/B$

In [ ]:
# Ax + By + C form:
print(ldaclf.coef_) # [A, B]
print(ldaclf.intercept_) # [C]

In [ ]:
# Show slope & intercept of decision plane line for lda
print('Slope = %0.2f'%(- ldaclf.coef_[0][0]/ldaclf.coef_[0][1]))
print('Intercept = %0.2f'%(- ldaclf.intercept_[0]/ldaclf.coef_[0][1]))

In [ ]:
# Show slope & intercept of decision plane line for svm
print('Slope = %0.2f'%(- svmclf.coef_[0][0]/svmclf.coef_[0][1]))
print('Intercept = %0.2f'%(- svmclf.intercept_[0]/svmclf.coef_[0][1]))

Plot decision boundary

In [ ]:
fig, ax = plt.subplots()
plot_classes(Xt[Yt==1], Xt[Yt==2], classifier=svmclf, clfcolor='c-', ax=ax)
plot_classes(Xt[Yt==1], Xt[Yt==2], classifier=ldaclf, clfcolor='m-', plot_data=False, ax=ax)

In [ ]:
fig, ax = plt.subplots()
plot_classes(Xv[Yv==1], Xv[Yv==2], classifier=svmclf, clfcolor='c-', ax=ax)
plot_classes(Xv[Yv==1], Xv[Yv==2], classifier=ldaclf, clfcolor='m-', plot_data=False, ax=ax)

> How would you change the data to make the classifier more or less accurate? 

In [ ]:
# Answer



# MVPA on real fMRI data
Now, let's try this on something more interesting - actual fMRI data. We will analyze the data from a simple localizer experiment, and try to classify which type of stimulus is present for each TR of a withheld run. 

First, we have to define our `X` and `Y` variables. NOTE that here, X is voxels! Up to now, Y has been voxel activity, but in this analysis, the direction of fitting is backwards with respect to regression - we are using *voxel activity* to predict *stimulus class* rather than stimulus class (or stimulus features, or experimental condition) to predict voxel activity.

Thus, X will be voxels. Our first decision is WHICH voxels to include. We will NOT do a full-brain analysis, here - we will select a large region of interest from combining pre-existing ROIs, using pycortex. 

In [ ]:
subject = 's03'
transform = 'category_localizer'
roi_masks = cx.get_roi_masks('s03', 'category_localizer', roi_list=['V1','V2','V3','V4','LO','OFA','FFA','EBA',
                                                                   'PPA','RSC','OPA'])
all_masks = np.array(list(roi_masks.values()))
print(all_masks.shape)
mask = np.any(all_masks, axis=0)
print(mask.shape)
#cx.webgl.show(cx.Volume(mask, subject, transform))

> Show the mask using pycortex! Which voxels are selected? Did we do this right?

In [ ]:
# Answer



Now, we will load 6 runs of localizer data, masking each run with the mask we have created to select only the voxels we care about

In [ ]:
fdir = '/unrshare/LESCROARTSHARE/IntroToEncodingModels/'
fbase = os.path.join(fdir, 's03_catloc_run%02d.nii.gz')
data = []
for run in range(1, 7):
    nii = nibabel.load(fbase%run)
    # Transpose at load time to make the data [t, z, y, x]
    tmp = nii.get_data().T
    # Mask the data to select only the voxels we care about
    data.append(tmp[:, mask])

> How would you create training and testing X data from this list of arrays?


In [ ]:
# Answer


Now we need Y, the labels for classes. This will come from the design matrix of the localizer experiment (formerly known as X,for regression!). Each class will be the type of stimulus shown in each block of the experiment (faces, places, bodies, etc). 

In [ ]:
with h5py.File(os.path.join(fdir, 'catloc_design.hdf')) as hf:
    print(list(hf.keys()))
    X = hf['X'].value
    xnames = hf['xnames'].value.tolist()
    # Ignore the 'decode' for now, it has to do with the format in which the strings were stored in this 
    # file, and it's just confusing...
    class_names = ['null'] + [x.decode() for x in xnames]
    events = hf['events'].value

> What are these variables? 

In [ ]:
# Answer
# (Display each variable, figure out what each is!)

Now we need Y variables. Which of these variables should be Y? 
> Define Y (The classes of the stimuli!) separately for training (Yt) and testing or validation (Yv)

In [ ]:
# Answer 


> Now, classify away! (Use the same sklearn classifiers we used above to try to classify the testing stimuli. Check how accurate your answers are!

In [ ]:
# Answer


Sklearn gives you a nice way to make confusion matrices, too:

In [ ]:
from sklearn import metrics

In [ ]:
cmatrix = metrics.confusion_matrix(Yv, Yv_pred)

# Show matrix
fig, ax = plt.subplots()
im = ax.imshow(cmatrix)
ax.set_xticks(np.arange(6))
ax.set_yticks(np.arange(6))
ax.set_xticklabels(class_names, rotation=90)
ax.set_yticklabels(class_names)
plt.colorbar(im)

## Now, let's show the classifier weights in pycortex!

> Where are the weights stored? 

(Remember that we are doing one-vs-all classifications, so there will be different weights to classify each class)

In [ ]:
# Answer


> How might you make your classification accuracy better? 

In [ ]:
# Answer